In [2]:
import numpy as np
import pandas as pd

In [1]:
from pybrain.datasets import ClassificationDataSet
from pybrain.utilities import percentError
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.structure.modules import SoftmaxLayer

In [9]:
#Reading in the training dataset
train = pd.read_csv('../outputs/features_v1.csv')
classes = train['class']
train_ids = train['id']
train = train[train.columns[1:-2]]

In [18]:
num_classes = len(np.unique(classes))
trndata = ClassificationDataSet(106, 1, nb_classes=num_classes)
for index, row in train.iterrows():
    trndata.addSample(np.array(row), classes[index])
trndata._convertToOneOfMany()

In [62]:
fnn = buildNetwork(trndata.indim, 500, trndata.outdim, outclass=SoftmaxLayer )

In [63]:
trainer = BackpropTrainer( fnn, dataset=trndata, momentum=0.1, verbose=True, weightdecay=0.01)

In [64]:
for i in range(500):
    trainer.trainEpochs(1)
    trnresult = percentError(trainer.testOnClassData(), trndata['class'])
    print "epoch: %4d" % trainer.totalepochs, \
          "train error: %5.2f%%" % trnresult,

Total error: 0.0167662501463
epoch:    1 train error: 21.39% Total error: 0.0130942783343
epoch:    2 train error: 18.96% Total error: 0.0114996679427
epoch:    3 train error: 28.16% Total error: 0.0117516664616
epoch:    4 train error: 20.74% Total error: 0.0122070071564
epoch:    5 train error: 23.91% Total error: 0.0133173317882
epoch:    6 train error: 26.41% Total error: 0.013678620325
epoch:    7 train error: 30.43% Total error: 0.0149365899051
epoch:    8 train error: 29.91% Total error: 0.014785937568
epoch:    9 train error: 25.96% Total error: 0.0151741680391
epoch:   10 train error: 24.76% Total error: 0.0151103133005
epoch:   11 train error: 23.98% Total error: 0.0139132033562
epoch:   12 train error: 32.57% Total error: 0.0140284740486
epoch:   13 train error: 38.92% Total error: 0.0138196909516
epoch:   14 train error: 24.56% Total error: 0.0137585766838
epoch:   15 train error: 23.36% Total error: 0.0135086499953
epoch:   16 train error: 24.76% Total error: 0.01373462704

In [65]:
#Loading the test data
test = pd.read_csv('../outputs/features_test_v1.csv')
test_ids = test['Id']
test = test[test.columns[1:-1]]

In [66]:
tstdata = ClassificationDataSet(106, 1, nb_classes=num_classes)
for index, row in test.iterrows():
    tstdata.addSample(np.array(row), 0)
tstdata._convertToOneOfMany()

In [67]:
outputs = np.array([fnn.activate(x) for x,_ in tstdata])

In [68]:
out = outputs.argmax(axis=1)

In [69]:
#combining to form a dataframe
test_df = pd.DataFrame(test_ids)
test_df['Prediction'] = out
test_df = test_df.set_index('Id')

In [70]:
test_df.to_csv('../outputs/neural_network_1.csv')